In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
SCORE_DIR = 'Downloads/data/scores'
box_scores = os.listdir(SCORE_DIR)

In [4]:
len(box_scores)

10285

In [5]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [6]:
#take in a box score
def parse_html(box_score):
    #read boxscore
    with open(box_score) as f:
        html = f.read()
    
    #initialize a soup instance
    soup = BeautifulSoup(html)
    #listcomp to select all table over header tags and deletes them
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')] #for the table header
    return soup

In [7]:
def read_line_score(soup):
    #attrs (attributes) specifies what we want to look at
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    
    #getting rid of indiv quarte scores
    line_score = line_score[['team', 'total']]
    return line_score

In [8]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors = 'coerce')
    return df

In [9]:
def read_season_info(soup):
    nav = soup.select('#bottom_nav_container')[0]
    hrefs = [a['href'] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split('_')[0]
    return season

In [10]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score['team'])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + '_max'

        summary = pd.concat([totals, maxes])
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if 'bpm' not in b]

        summary = summary[base_cols]
        summaries.append(summary)

    summary = pd.concat(summaries, axis = 1).T

    game = pd.concat([summary, line_score], axis =1)
    game['home'] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += '_opp'

    full_game = pd.concat([game, game_opp], axis =1)

    full_game['season'] = read_season_info(soup)
    full_game['date'] = os.path.basename(box_score)[:8]
    full_game['date'] = pd.to_datetime(full_game['date'], format = '%Y%m%d')
    full_game['won'] = full_game['total'] > full_game['total_opp']

    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 10284
200 / 10284
300 / 10284
400 / 10284
500 / 10284
600 / 10284
700 / 10284
800 / 10284
900 / 10284
1000 / 10284
1100 / 10284
1200 / 10284
1300 / 10284
1400 / 10284
1500 / 10284
1600 / 10284
1700 / 10284
1800 / 10284
1900 / 10284
2000 / 10284
2100 / 10284
2200 / 10284
2300 / 10284
2400 / 10284
2500 / 10284
2600 / 10284
2700 / 10284
2800 / 10284
2900 / 10284
3000 / 10284
3100 / 10284
3200 / 10284
3300 / 10284
3400 / 10284
3500 / 10284
3600 / 10284
3700 / 10284
3800 / 10284
3900 / 10284
4000 / 10284
4100 / 10284
4200 / 10284
4300 / 10284
4400 / 10284
4500 / 10284
4600 / 10284
4700 / 10284
4800 / 10284
4900 / 10284
5000 / 10284
5100 / 10284
5200 / 10284
5300 / 10284
5400 / 10284
5500 / 10284
5600 / 10284
5700 / 10284
5800 / 10284
5900 / 10284
6000 / 10284
6100 / 10284
6200 / 10284
6300 / 10284
6400 / 10284
6500 / 10284
6600 / 10284
6700 / 10284
6800 / 10284
6900 / 10284
7000 / 10284
7100 / 10284
7200 / 10284
7300 / 10284
7400 / 10284
7500 / 10284
7600 / 10284
7700 / 10284
7800 / 1

In [11]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
games_df

In [ ]:
games_df.to_csv('nba_games.csv')